In [40]:
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [41]:
# Environment and utilities
import os
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool, OpenAIServerModel

model = OpenAIServerModel(
    model_id="qwen2.5-coder-32b-instruct",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.environ["DASHSCOPE_API_KEY"],
)

In [ ]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [ ]:
# tools=[GoogleSearchTool("serper"), VisitWebpageTool(), calculate_cargo_travel_time]
tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time]

agent = CodeAgent(
    model=model,
    tools=tools,
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [ ]:
result = agent.run(task)
result

In [ ]:
result

In [ ]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

In [50]:
webagent_model = OpenAIServerModel(
    model_id="qwen2.5-coder-32b-instruct",
    api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=os.environ["DASHSCOPE_API_KEY"],
    max_tokens=8096
)
web_agent = CodeAgent(
    model=webagent_model,
    tools=[
        # DuckDuckGoSearchTool(),
        GoogleSearchTool("serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [51]:
from smolagents.utils import encode_image_base64, make_image_url

def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel(
        model_id="qwen-omni-turbo",
        api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        api_key=os.environ["DASHSCOPE_API_KEY"],
        max_tokens=8096
        )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True

manager_model = OpenAIServerModel(
    model_id="gemini-2.0-flash",
    api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=os.getenv('GOOGLE_API_KEY'),
    max_tokens=8096
)
manager_agent = CodeAgent(
    model=manager_model,
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly", "plotly.express",
        "shapely",
        "json",
        "pandas",
        "numpy"
],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [52]:
manager_agent.visualize()

CodeAgent | gemini-2.0-flash
├── ✅ Authorized imports: ['geopandas', 'plotly', 'plotly.express', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | qwen2.5-coder-32b-instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance

In [ ]:
# original task for the manager agent
# manager_agent.run("""
# Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
# Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
# Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

# Here's an example of how to plot and return a map:
# import plotly.express as px
# df = px.data.carshare()
# fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
#      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
# fig.show()
# fig.write_image("saved_image.png")
# final_answer(fig)

# Never try to process strings using code: when you have a string to read, just print it and you'll see it.
# """)

In [53]:
# updated task avoid using mapbox token
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_geo(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", 
                     color_continuous_scale=px.colors.sequential.Magma, size_max=15,
                     projection="natural earth")
fig.update_traces(marker=dict(size=10))
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_geo(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",                │
│                      color_continuous_scale=px.colors.sequential.Magma, size_max=15,                            │
│                      projection="natural earth")                                                                │
│ fig.update_traces(marker=dict(size=10))                                                                         │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ OpenAIServerModel - gemini-2.0-flash ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task

*   My location: Gotham (40.7128° N, 74.0060° W)
*   Task: Find all Batman filming locations in the world.
*   Task: Calculate the time to transfer via cargo plane from each Batman filming location to Gotham.
*   Task: Find supercar factories with similar cargo plane transfer times to those from Batman locations.
*   Task: Need at least 6 locations in total.
*   Task: Represent this as spatial map of the world, with the locations represented as scatter points with a color
that depends on the travel time, and save it to saved_map.png.

### 1.2. Facts to look up

*   List of Batman filming locations: Use a web search to find a comprehensive list of filming locations for Batman
movies. Search query: "Batman filming locations". We need to identify the latitude and longitude coordinates for 
each location.
*   Supercar factory locations: Use a web search to find a list of supercar factories and their corresponding 
locations (latitude and longitude). Search query: "supercar factory locations".

### 1.3. Facts to derive

*   Travel time from each Batman filming location to Gotham: Use the given coordinates for Gotham and the 
coordinates obtained from the web search for each filming location as input to the `calculate_cargo_travel_time` 
function.
*   Travel time from each supercar factory to Gotham: Use the given coordinates for Gotham and the coordinates 
obtained from the web search for each supercar factory as input to the `calculate_cargo_travel_time` function.
*   Color scale for the map: Based on the range of travel times calculated, determine an appropriate color scale 
for the scatter plot.

## 2. Plan

1.  **Find Batman filming locations:** Call the `web_agent` to find a list of Batman filming locations and their 
coordinates. The task should be: "Find a list of Batman filming locations with their latitude and longitude 
coordinates. Include filming locations from all Batman movies (e.g., Batman Begins, The Dark Knight, etc.). Return 
the information in a structured format, if possible, such as a table or list."
2.  **Find Supercar factory locations:** Call the `web_agent` to find a list of supercar factories and their 
coordinates. The task should be: "Find a list of supercar factories with their latitude and longitude coordinates. 
Prioritize factories that manufacture well-known supercars (e.g., Ferrari, Lamborghini, McLaren, etc.). Return the 
information in a structured format, if possible, such as a table or list."
3.  **Process the filming locations list:** Parse the string output from the web search, extracting location names,
latitudes, and longitudes of Batman filming locations.
4.  **Process the factories locations list:** Parse the string output from the web search, extracting location 
names, latitudes, and longitudes of Supercar factories.
5.  **Calculate travel times for each Batman filming location:** Iterate through the list of Batman filming 
locations. For each location, call the `calculate_cargo_travel_time` function with the filming location coordinates
and Gotham's coordinates to calculate the travel time. Store the location name, coordinates, and travel time.
6.  **Calculate travel times for each Supercar factory:** Iterate through the list of Supercar factory. For each 
location, call the `calculate_cargo_travel_time` function with the factory coordinates and Gotham's coordinates to 
calculate the travel time. Store the factory name, coordinates, and travel time.
7.  **Filter Supercar factories by travel time:** Compare the travel times of the supercar factories to the travel 
times of the Batman filming locations. Keep only the supercar factories that have similar travel times. Define a 
threshold for "similar" travel times (e.g., within 1 ho

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I will start by finding the Batman filming locations and the supercar factory locations using the         
`web_agent` tool. This will give me the information I need to calculate travel times and plot the locations on a   
map.                                                                                                               
Code:                                                                                                              
```py                                                                                                              
batman_task = "Find a list of Batman filming locations with their latitude and longitude coordinates. Include      
filming locations from all Batman movies (e.g., Batman Begins, The Dark Knight, etc.). Return the information in a 
structured format, if possible, such as a table or list."                                                          
supercar_task = "Find a list of supercar factories with their latitude and longitude coordinates. Prioritize       
factories that manufacture well-known supercars (e.g., Ferrari, Lamborghini, McLaren, etc.). Return the information
in a structured format, if possible, such as a table or list."                                                     
                                                                                                                   
batman_locations_info = web_agent(batman_task)                                                                     
print("Batman locations:\n", batman_locations_info)                                                                
                                                                                                                   
supercar_factories_info = web_agent(supercar_task)                                                                 
print("Supercar factories:\n", supercar_factories_info)                                                            
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_task = "Find a list of Batman filming locations with their latitude and longitude coordinates. Include    
  filming locations from all Batman movies (e.g., Batman Begins, The Dark Knight, etc.). Return the information    
  in a structured format, if possible, such as a table or list."                                                   
  supercar_task = "Find a list of supercar factories with their latitude and longitude coordinates. Prioritize     
  factories that manufacture well-known supercars (e.g., Ferrari, Lamborghini, McLaren, etc.). Return the          
  information in a structured format, if possible, such as a table or list."                                       
                                                                                                                   
  batman_locations_info = web_agent(batman_task)                                                                   
  print("Batman locations:\n", batman_locations_info)                                                              
                                                                                                                   
  supercar_factories_info = web_agent(supercar_task)                                                               
  print("Supercar factories:\n", supercar_factories_info)                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Check check_reasoning_and_plot failed with error: maximum recursion depth exceeded

[Step 15: Duration 2.79 seconds| Input tokens: 298,579 | Output tokens: 7,777]

Reached max steps.

[Step 16: Duration 3.73 seconds| Input tokens: 326,319 | Output tokens: 7,791]

'I am unable to generate the requested map due to environment limitations.\n'